# Physics-Informed Neural Networks (PINNs) with Fourier Feature Mapping

This notebook implements a Physics-Informed Neural Network (PINN) to solve partial differential equations (PDEs) using Fourier feature mapping. The notebook uses PyTorch for defining and training the network.

```python

# Import necessary libraries

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import grad
import torch.functional as F
import numpy as np
import matplotlib.pyplot as plt


# Define the Fourier Feature Mapping Class


In [ ]:
class FFM(nn.Module):
    def __init__(self, in_dim, out_dim, std_dev=2):
        """
        Initializes the Fourier Feature Mapping.
        
        Parameters:
        in_dim (int): Input dimensions (number of independent variables)
        out_dim (int): Output dimensions (length of the hidden layer)
        std_dev (float): Standard deviation for initializing omega parameter
        """
        super().__init__()
        self.omega = nn.Parameter(torch.randn(out_dim, in_dim) * std_dev)

    def forward(self, x):
        """
        Forward pass through the Fourier feature mapping layer.
        
        Parameters:
        x (Tensor): Input tensor
        
        Returns:
        Tensor: Output tensor after applying cosine function
        """
        return torch.cos(F.F.linear(x, self.omega))


# Define the PINNs Network Class

In [ ]:
class PINNsNet(nn.Module):
    def __init__(self, in_dim=2, HL_dim=32, out_dim=1, activation=nn.Tanh()):
        """
        Initializes the PINNs network.
        
        Parameters:
        in_dim (int): Input dimensions (number of independent variables)
        HL_dim (int): Width of the network (number of hidden layer units)
        out_dim (int): Output dimensions (number of dependent variables)
        activation (nn.Module): Activation function to use in the network
        """
        super().__init__()
        
        # Define the network architecture
        network = [nn.Linear(in_dim, HL_dim), activation,
                   nn.Linear(HL_dim, HL_dim), activation,
                   nn.Linear(HL_dim, HL_dim), activation,
                   nn.Linear(HL_dim, HL_dim), activation,
                   nn.Linear(HL_dim, out_dim)]
        
        # Define the network using the sequential method
        self.u = nn.Sequential(*network)
    
    def forward(self, x, t):
        """
        Forward pass through the network.
        
        Parameters:
        x (Tensor): Spatial coordinates
        t (Tensor): Temporal coordinates
        
        Returns:
        Tensor: Network output
        """
        return self.u(torch.cat((x, t), 1))
    
    def compute_loss(self, x, t, Nx, Nt):
        """
        Computes the loss for the network.
        
        Parameters:
        x (Tensor): Spatial coordinates with gradient tracking enabled
        t (Tensor): Temporal coordinates with gradient tracking enabled
        Nx (int): Number of spatial points
        Nt (int): Number of temporal points
        
        Returns:
        tuple: Losses for PDE, boundary conditions, and initial conditions
        """
        x.requires_grad = True
        t.requires_grad = True
        u = self.u(torch.cat((x, t), 1))

        # Compute PDE derivatives using auto-grad
        u_t = grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_x = grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_xx = grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
        
        # Define a loss function
        loss_fun = nn.MSELoss()

        # Compute the PDE residual loss
        res = u_t - 0.1 * u_xx
        pde_loss = loss_fun(res, torch.zeros_like(res))

        # Compute the boundary condition (BC) loss
        u_reshaped = u.view(Nx, Nt)
        u_x_reshaped = u_x.view(Nx, Nt)
        bc_loss = (loss_fun(u_reshaped[0, :], torch.zeros_like(u_reshaped[0, :])) +
                   loss_fun(u_reshaped[Nx-1, :], torch.zeros_like(u_reshaped[Nx-1, :])) +
                   loss_fun(u_x_reshaped[0, :], u_x_reshaped[Nx-1, :]))
        
        # Compute the initial condition (IC) loss
        x_reshaped = x.view(Nx, Nt)
        u_initial = torch.sin(2 * np.pi * x_reshaped[:, 0])
        ic_loss = loss_fun(u_initial, u_reshaped[:, 0])
    
        return pde_loss, bc_loss, ic_loss


### Define Model and Optimizer

In [ ]:
model = PINNs_net()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# num of points in the domain
Nx, Nt = 128, 128

# define domain dimensions and resolution
Lx_initial, Lx_final = 0, 1
t_initial, t_final = 0, 1
dx = (Lx_final - Lx_initial) / (Nx - 1)
dt = (t_final - t_initial) / (Nt-1)

# initiallize input parameters as tensors
x = torch.zeros(Nx, Nt)
t = torch.zeros(Nx, Nt)
for i in range(Nx):
    for j in range(Nt):
        x[i,j] = Lx_initial + dx * i
        t[i,j] = t_initial + dt * j


In [ ]:
for epoch in range(500):
    # compute various losses
    eq_loss, BC_loss, IC_loss = model.compute_loss(x.view(-1,1), t.view(-1,1), Nx, Nt)

    # compute total loss
    total_loss = eq_loss + 20*BC_loss + 20*IC_loss

    # backward pass
    total_loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    print(f"epoch: {epoch}, loss: {total_loss}")